In [ ]:
import pandas as pd
from Project2.Code.perceptron import Perceptron
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import numpy as np
data=load_digits(as_frame=True);
data=data.frame
data0=data[data['target']==0]
data1=data[data['target']==1]
data=pd.concat([data0,data1])
data.values

In [ ]:
Xdata,Xtest,ydata,ytest=train_test_split(data.values[:,0:-1],data.values[:,-1],test_size=0.2,random_state=0)
Xtrain,Xval,ytrain,yval=train_test_split(Xdata,ydata,test_size=0.2,random_state=0)
perceptron=Perceptron()
perceptron.train(Xtrain,ytrain)
print(np.mean(perceptron.predict(Xval)==yval))